# Test biaslyze with the toxic comments dataset

Data source: https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/tobias/Repositories/biaslyze/')

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

## Load and prepare data

In [4]:
df = pd.read_csv("../data/jigsaw-toxic-comment-classification/train.csv"); df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
# make the classification problem binary
df["target"] = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].sum(axis=1) > 0

## Train a BoW-model

In [6]:
clf = make_pipeline(TfidfVectorizer(min_df=10, max_features=10000, stop_words="english"), LogisticRegression())

In [7]:
clf.fit(df.comment_text, df.target)

/home/tobias/.cache/pypoetry/virtualenvs/biaslyze-OjqpgMw8-py3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=10000, min_df=10,
                                 stop_words='english')),
                ('logisticregression', LogisticRegression())])

In [8]:
train_pred = clf.predict(df.comment_text)
print(accuracy_score(df.target, train_pred))

0.9605755431751384


### probe the model a bit to see if we got an obvious problem

In [9]:
test_text = "Jews and muslims are great friends!"

print(clf.predict_proba([test_text]))

[[0.91499231 0.08500769]]


In [10]:
test_text = "That woman is great!"

print(clf.predict_proba([test_text]))

[[0.9287197 0.0712803]]


In [11]:
test_text = "That man is great!"

print(clf.predict_proba([test_text]))

[[0.92822178 0.07177822]]


In [12]:
test_text = "That muslim woman is great deed for islam!"

print(clf.predict_proba([test_text]))

[[0.79439839 0.20560161]]


In [43]:
test_text = "My mother is a jew."

print(clf.predict_proba([test_text]))

[[0.27918978 0.72081022]]


### Look at the logistc regression coefficients

In [13]:
coef = clf.named_steps.get("logisticregression").coef_

In [14]:
vocab = clf.named_steps.get("tfidfvectorizer").vocabulary_

In [15]:
vocab_coef_dict = {token: coef[0][idx] for token, idx in vocab.items()}

In [16]:
dict(sorted(vocab_coef_dict.items(), key=lambda v: -v[1]))

{'fuck': 17.80540365099103,
 'fucking': 14.635332881375062,
 'shit': 12.776545770068545,
 'idiot': 12.575476050602957,
 'stupid': 11.43395617454563,
 'ass': 10.928078279290391,
 'bullshit': 10.013142053233421,
 'asshole': 9.934778096130305,
 'bitch': 9.418717740305228,
 'suck': 9.310144753582346,
 'crap': 8.497103499739891,
 'dick': 7.980834777990305,
 'moron': 7.977000019330612,
 'cunt': 7.970742269355696,
 'faggot': 7.927285379438609,
 'idiots': 7.5557181070319865,
 'sucks': 7.55191693535309,
 'penis': 7.131695708016156,
 'bastard': 6.814701495629582,
 'pathetic': 6.797036943528691,
 'hell': 6.54890040717585,
 'fucked': 6.547838398426092,
 'jerk': 6.502470382314543,
 'dumb': 5.974766115964371,
 'cock': 5.953124807623002,
 'nigger': 5.916553918658378,
 'gay': 5.90417015335498,
 'dumbass': 5.838005903526412,
 'shut': 5.795057292104361,
 'loser': 5.78579132509518,
 'fag': 5.763386768266206,
 'idiotic': 5.711392879783656,
 'pussy': 5.602145220574613,
 'piss': 5.593026077075599,
 'fucker'

## Test LIME based bias detection with keywords

In [32]:
from biaslyze.evaluators import LimeBiasEvaluator
from biaslyze.bias_detectors import LimeKeywordBiasDetector

In [33]:
bias_detector = LimeKeywordBiasDetector(
    bias_evaluator=LimeBiasEvaluator(n_lime_samples=1000),
    n_top_keywords=30,
    use_tokenizer=True
)

In [34]:
test_texts = df.comment_text.sample(10000)
detection_res = bias_detector.detect(texts=test_texts, predict_func=clf.predict_proba)

2023-04-27 11:56:41.293 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 10000 texts...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:56<00:00, 56.52it/s]
2023-04-27 11:59:38.226 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 3974 texts with protected concepts.
2023-04-27 11:59:38.227 | INFO     | biaslyze.evaluators:evaluate:44 - Started bias detection on 3974 samples...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3974/3974 [37:59<00:00,  1.74it/s]


In [35]:
detection_res.summary()

Detected 2821 samples with potential issues.
    Potentially problematic concepts detected: [('gender', 2117), ('nationality', 760), ('religion', 146), ('gendered words', 62)]
    Based on keywords: [('they', 498), ('he', 412), ('his', 326), ('them', 286), ('their', 220), ('him', 169), ('man', 106), ('her', 85), ('she', 82), ('guy', 60), ('guys', 52), ('god', 51), ('american', 44), ('german', 44), ('uk', 38), ('himself', 32), ('french', 31), ('christian', 29), ('jewish', 28), ('india', 27)].


In [36]:
detection_res.details(group_by_concept=True)

Concept: gender
[{'reason': ['they', 'them', 'he'],
  'text': '"\n'
          '\n'
          ' Goethe \n'
          '\n'
          '""All truly wise thoughts have been thought already thousands of '
          'times; but to make them truly ours, we must think them over again '
          'honestly, till they take root in our personal experience."" – '
          'Johann Wolfgang von Goethe\n'
          '\n'
          'I don\'t think he is referring to eternal return here.  "'},
 {'reason': ['afghan', 'their'],
  'text': '"Pashtun Political Party ""Afghan Mellat"" and their agenda "'},
 {'reason': ['he'],
  'text': "Ok then, but Joe the Plumber got a lot more, why isn't he "
          'mentioned? Why Ayers?'},
 {'reason': ['they'],
  'text': 'Question from an amateur: are AVL trees generally considered worse '
          '(in terms of performance) than red-black or splay trees?  Also, are '
          'they easier to implement?  I suspect both to be the case.  If so, '
          'someone sh

In [37]:
test_df = df.iloc[test_texts.index]

In [38]:
test_df[test_df.target].reset_index().comment_text[4]

"Blimey! ... \n\n... you really are a sad , fat sack of Sh1t, aren't you?"

In [40]:
from bokeh.io import show, output_notebook

output_notebook()

dashboard = detection_res.dashboard()
show(dashboard)

Loading BokehJS ...

In [42]:
pos_dashboard = detection_res.dashboard(use_position=True)
show(pos_dashboard)

## Try to cluster detected biases

In [44]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

In [45]:
# Prepare embeddings
docs = [sample.text for sample in detection_res.biased_samples]
y = [", ".join(sample.bias_concepts) for sample in detection_res.biased_samples]
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
vectorizer_model = CountVectorizer(stop_words="english")
embeddings = sentence_model.encode(docs, show_progress_bar=False)

hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', max_cluster_size=25)
# Train BERTopic
topic_model = BERTopic(hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model).fit(docs, embeddings)

IndexError: list index out of range

In [ ]:
# Run the visualization with the original embeddings
topic_model.visualize_documents(docs, embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)

In [ ]:
topic_model.topic_labels_

In [ ]:
import yaml
from bokeh.themes import Theme
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Slider, TextInput, HoverTool
from bokeh.palettes import Spectral5, Accent5, Category10_10
from bokeh.io import curdoc
from bokeh.layouts import column, row


def bkapp(doc):
    hover_tool = HoverTool(tooltips=[("Text", "@text"), ("Keywords", "@keywords")])
    p = figure(width=1200, height=800, tools=["pan","wheel_zoom","box_zoom","reset", hover_tool])

    bias_concept = [",".join(sample.bias_concepts) for sample in detection_res.biased_samples]
    concept2int = list(set(bias_concept))
    num_bias_concept = [concept2int.index(c) for c in bias_concept]
    bias_intensity = [50*np.abs(sample.score) for sample in detection_res.biased_samples]

    # configure
    df = pd.DataFrame(dict(
        text=[sample.text for sample in detection_res.biased_samples],
        keywords=[",".join(sample.bias_reasons) for sample in detection_res.biased_samples],
        x=reduced_embeddings[:,0],
        y=reduced_embeddings[:,1],
        color=[Category10_10[idx] for idx in num_bias_concept],
        bias_intensity=bias_intensity,
        bias_concept=bias_concept,
    ))
    source = ColumnDataSource(data=df)

    # add a circle renderer with a size, color, and alpha
    p.scatter(
        "x", "y",
        source=source,
        color="color",
        size="bias_intensity",
        alpha=0.3,
        legend_group="bias_concept",
    )

    p.legend.location = "top_left"
    #p.legend.click_policy="hide"

    # slider
    threshold = Slider(title="threshold", value=0.0, start=0.0, end=1.0, step=0.01, width=750)

    def update_data(attrname, old, new):
        # Get the current slider values
        t = threshold.value
        new_df = df.copy()
        new_df["bias_intensity"] = new_df.bias_intensity.apply(lambda x: x if x >= 50*t else 0.0)
        source.data = new_df

    threshold.on_change('value', update_data)


    doc.add_root(column(threshold, p, width=800))
    # show the results
    doc.theme = Theme(json=yaml.load("""
        attrs:
            figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
                height: 800
                width: 1200
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

show(bkapp)

## Testing a sentiment analysis model from huggingface

In [ ]:
from transformers import pipeline
from torch.utils.data import Dataset


classifier = pipeline(
    model="distilbert-base-uncased-finetuned-sst-2-english",
    top_k=None,
    padding=True,
    truncation=True
)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]


def predict_sentiment(texts):
    data = MyDataset(texts)
    proba = []
    for res in classifier(data):
        proba_array = []
        for p in sorted(res, key=lambda d: d['label'], reverse=True):
            proba_array.append(p.get("score"))
        proba.append(np.array(proba_array))
    return np.array(proba) / np.array(proba).sum(axis=1)[:,None]

In [ ]:
bias_detector = LimeKeywordBiasDetector(
    bias_evaluator=LimeBiasEvaluator(n_lime_samples=500),
    n_top_keywords=10,
    use_tokenizer=True
)

In [ ]:
test_texts = df.comment_text.sample(50)
detection_res = bias_detector.detect(texts=test_texts, predict_func=predict_sentiment)

In [ ]:
detection_res.summary()

In [ ]:
detection_res.details(group_by_concept=True)

## !! Very Experimental !!: Test masked language model based bias detection with keywords

In [ ]:
from biaslyze.bias_detectors import MaskedKeywordBiasDetector
from biaslyze.concept_detectors import KeywordConceptDetector

In [ ]:
bias_detector = MaskedKeywordBiasDetector(n_resample_keywords=20, use_tokenizer=True)

In [ ]:
detection_res = bias_detector.detect(
    texts=df.comment_text.sample(1000),
    predict_func=clf.predict_proba
)

In [ ]:
detection_res.summary()

In [ ]:
detection_res.details()

In [ ]:
from bokeh.io import show, output_notebook

output_notebook()

dashboard = detection_res.dashboard()
show(dashboard)